# Analisis Exploratorio de PTB XL

In [1]:
import numpy as np
import pandas as pd

### Vistazo a las tablas

Cargamos el csv con la metadata referente a las 21837 imagenes:

In [3]:
data = pd.read_csv('../data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv')

In [136]:
print(len(data))
data.head()

21837


,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


La columna _scp_codes_ contiene informacion respecto a la(s) [clase(s)](https://www.nature.com/articles/s41597-020-0495-6/tables/7) a la(s) que pertenece la imagen, y a la [forma](https://www.nature.com/articles/s41597-020-0495-6/tables/8), y al [ritmo](https://www.nature.com/articles/s41597-020-0495-6/tables/9) de las señales.

Al momento aun no sabemos por que hay valores numericos en los diccionarios. Por ejemplo, en el indice 0 vs 1, ¿qué implica que NORM tenga un valor de 100.0 en el primero y de 80.0 en el segundo? O, ¿por qué en el primero los valores de LVOLT y de SR son 0.0 y 0.0? Por el momento solo nos fijaremos en el texto y no en el valor numerico.

In [141]:
data['scp_codes'].head()

0    {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                {'NORM': 80.0, 'SBRAD': 0.0}
2                  {'NORM': 100.0, 'SR': 0.0}
3                  {'NORM': 100.0, 'SR': 0.0}
4                  {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, dtype: object

Para ser mas especificos, los codigos son de las subsubclases. En este [link](https://www.nature.com/articles/s41597-020-0495-6/tables/7) se encuentra una tabla con el mapeo de estas subsubclases hacia las subclases (en la tabla es _Subclass_) y las clases (en la tabla es _Supclass_).

Las tablas de clase, forma y ritmo no estan contenidas en el .zip, pero facilmente se pueden copiar y pegar a un csv para guardarlas localmente y posteriormente hacer los mapeos necesarios

In [76]:
class_df = pd.read_csv('../data/ptb_xl_class_dictionary.csv')
form_df = pd.read_csv('../data/ptb_xl_form_dictionary.csv')
rhythm_df = pd.read_csv('../data/ptb_xl_rhythm_dictionary.csv')

In [137]:
class_df.head()

,Class,# Records,Description,Superclass,Subclass
0,LAFB,1626,left anterior fascicular block,CD,LAFB/LPFB
1,IRBBB,1118,incomplete right bundle branch block,CD,IRBBB
2,AVB,797,first degree AV block,CD,_AVB
3,IVCD,789,non-specific intraventricular conduction distu...,CD,IVCD
4,CRBBB,542,complete right bundle branch block,CD,CRBBB


In [138]:
form_df.head()

,Form,# Records,Description
0,NDT,1829,non-diagnostic T abnormalities
1,NST_,770,non-specific ST changes
2,DIG,181,digitalis-effect
3,LNGQT,118,long QT-interval
4,ABQRS,3327,abnormal QRS


In [139]:
rhythm_df.head()

,Rhythm,# Records,Description
0,SR,16782,sinus rhythm
1,AFIB,1514,atrial fibrillation
2,STACH,826,sinus tachycardia
3,SARRH,772,sinus arrhythmia
4,SBRAD,637,sinus bradycardia


### Subsubclases a clases

Trabajamos entonces con la columna _scp_codes_ para recuperar las clases a partir de las subsubclases.

In [142]:
### Aplicamos eval pues el formato venia como dict en un string
scp_codes = data['scp_codes'].apply(lambda x : eval(x))

### Hacemos un diccionario para mapear subsubclase a clase
class_dict = {key : value for key, value in zip(class_df['Class'], class_df['Superclass'])}

In [155]:
### Obtenemos los valores unicos de las subsubclases
class_set = set(class_df['Class'])

### Aqui guardaremos el mapeo de las clases a subsubclases, para
### cada etiqueta que en efecto corresponda a una subsubclase 
### (pues tambien podria ser una etiqueta de forma o de ritmo)
class_series = scp_codes.copy()

for k in range(len(superclass_series)):
    aux = set(scp_codes.iloc[k].keys()) # Etiquetas por registro
    aux = aux.intersection(class_set) # Conservamos solo las que son subsubclases
    aux = list(aux)

    classes_aux = []
    for class_ in aux:
        # Hacemos el mapeo de subsubclase a clase y lo guardamos
        classes_aux.append(class_dict[class_])

    class_series.iloc[k] = classes_aux

In [156]:
class_series.head()

0    [NORM]
1    [NORM]
2    [NORM]
3    [NORM]
4    [NORM]
Name: scp_codes, dtype: object

Es util tener los valores de las clases a las que pertenece cada registro, pero es aun mejor tener el OneHot-Encoding para hacer facilmente calculos de las intersecciones:

In [157]:
### Aqui guardaremos el df con el OHE de los registros vs las clases
ohe_class_df = pd.DataFrame(columns = list(class_df['Superclass'].unique()))

for k in range(len(superclass_series)):
    aux = class_series.iloc[k]
    class_aux = pd.DataFrame(index = [0], columns = list(class_df['Superclass'].unique()))

    for _ in range(len(aux)):
        ### Asignamos un 1 a las columnas referentes a cada clase por cada registro
        class_aux[aux[_]] = 1

    ohe_class_df = pd.concat([ohe_class_df, class_aux])

In [166]:
ohe_class_df = ohe_class_df.fillna(0) ### Llenamos de 0 los NaN
ohe_class_df.reset_index(inplace = True)
ohe_class_df.head()

,index,CD,HYP,MI,NORM,STTC
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


Con el OHE, es mas facil calcular cuantos registros contienen intersecciones de clases. Son de particular interes las clases NORM, MI y STTC:

In [168]:
val = 'MI'
print(f'Total {val} registers:', sum(ohe_class_df[val] == 1))
val = 'STTC'
print(f'Total {val} registers:', sum(ohe_class_df[val] == 1))
val = 'NORM'
print(f'Total {val} registers:', sum(ohe_class_df[val] == 1))

print()

val1, val2 = 'MI', 'STTC'
print(f'Total {val1} & {val2} registers:', sum((ohe_class_df[val1] == 1) & (ohe_class_df[val2] == 1)))
val1, val2 = 'MI', 'NORM'
print(f'Total {val1} & {val2} registers:', sum((ohe_class_df[val1] == 1) & (ohe_class_df[val2] == 1)))
val1, val2 = 'STTC', 'NORM'
print(f'Total {val1} & {val2} registers:', sum((ohe_class_df[val1] == 1) & (ohe_class_df[val2] == 1)))

Total MI registers: 5486
Total STTC registers: 5250
Total NORM registers: 9528

Total MI & STTC registers: 1345
Total MI & NORM registers: 1
Total STTC & NORM registers: 33


Podemos ver que la intereccion MI & STTC contiene un numero elevado de registros relativo a la cantidad de registros que cada clase tiene independientemente. Por otra parte, vemos que las intersecciones con NORM son muy pequeñas tanto con MI (solo 1, incluso quiza fue un error) como con STTC.

### Subsubclases a subclases

Trabajamos entonces con la columna _scp_codes_ para recuperar las subclases a partir de las subsubclases.

In [170]:
### Hacemos un diccionario para mapear subsubclase a subclase
subclass_dict = {key : value for key, value in zip(class_df['Class'], class_df['Subclass'])}

In [175]:
scp_codes.iloc[k].keys()

dict_keys(['NORM', 'SR'])

In [172]:
### Obtenemos los valores unicos de las subsubclases
class_set = set(class_df['Class'])

### Aqui guardaremos el mapeo de las subclases a subsubclases, para
### cada etiqueta que en efecto corresponda a una subsubclase 
### (pues tambien podria ser una etiqueta de forma o de ritmo)
subclass_series = scp_codes.copy()

for k in range(len(subclass_series)):
    aux = set(scp_codes.iloc[k].keys()) # Etiquetas por registro
    aux = aux.intersection(class_set) # Conservamos solo las que son subsubclases
    aux = list(aux)

    classes_aux = []
    for subclass_ in aux:
        # Hacemos el mapeo de subsubclase a clase y lo guardamos
        classes_aux.append(subclass_dict[subclass_])

    subclass_series.iloc[k] = classes_aux

In [173]:
subclass_series.head()

0    [NORM]
1    [NORM]
2    [NORM]
3    [NORM]
4    [NORM]
Name: scp_codes, dtype: object

Es util tener los valores de las subclases a las que pertenece cada registro, pero es aun mejor tener el OneHot-Encoding para hacer facilmente calculos de las intersecciones:

In [180]:
### Aqui guardaremos el df con el OHE de los registros vs las clases
ohe_subclass_df = pd.DataFrame(columns = list(class_df['Subclass'].unique()))

for k in range(len(subclass_series)):
    aux = subclass_series.iloc[k]
    class_aux = pd.DataFrame(index = [0], columns = list(class_df['Subclass'].unique()))

    for _ in range(len(aux)):
        ### Asignamos un 1 a las columnas referentes a cada clase por cada registro
        class_aux[aux[_]] = 1

    ohe_subclass_df = pd.concat([ohe_subclass_df, class_aux])

In [181]:
ohe_subclass_df = ohe_subclass_df.fillna(0) ### Llenamos de 0 los NaN
ohe_subclass_df.reset_index(inplace = True)
ohe_subclass_df.head()

,index,LAFB/LPFB,IRBBB,_AVB,IVCD,CRBBB,CLBBB,WPW,ILBBB,LVH,...,IMI,AMI,LMI,PMI,NORM,STTC,NST_,ISC_,ISCA,ISCI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [186]:
ohe_subclass_df.columns

Index(['index', 'LAFB/LPFB', 'IRBBB', '_AVB', 'IVCD', 'CRBBB', 'CLBBB', 'WPW',
       'ILBBB', 'LVH', 'LAO/LAE', 'RVH', 'RAO/RAE', 'SEHYP', 'IMI', 'AMI',
       'LMI', 'PMI', 'NORM', 'STTC', 'NST_', 'ISC_', 'ISCA', 'ISCI'],
      dtype='object')

Con el OHE, es mas facil calcular cuantos registros contienen intersecciones de clases. Son de particular interes las clases NORM, MI y STTC:

In [262]:
val = 'ISCI'
print(f'Total {val} registers:', sum(ohe_subclass_df[val] == 1))
val = 'ISCA'
print(f'Total {val} registers:', sum(ohe_subclass_df[val] == 1))
val = 'IMI'
print(f'Total {val} registers:', sum(ohe_subclass_df[val] == 1))
val = 'AMI'
print(f'Total {val} registers:', sum(ohe_subclass_df[val] == 1))

print()

val1, val2 = 'ISCI', 'ISCA'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))
val1, val2 = 'IMI', 'AMI'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))
val1, val2 = 'ISCI', 'IMI'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))
val1, val2 = 'ISCI', 'AMI'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))
val1, val2 = 'ISCA', 'IMI'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))
val1, val2 = 'ISCA', 'AMI'
print(f'Total {val1} & {val2} registers:', sum((ohe_subclass_df[val1] == 1) & (ohe_subclass_df[val2] == 1)))

Total ISCI registers: 398
Total ISCA registers: 944
Total IMI registers: 3281
Total AMI registers: 3086

Total ISCI & ISCA registers: 142
Total IMI & AMI registers: 996
Total ISCI & IMI registers: 7
Total ISCI & AMI registers: 68
Total ISCA & IMI registers: 242
Total ISCA & AMI registers: 246


Vemos que hay interescciones entre las subclases ISCI e ISCA, pertenecientes a la clase STTC. Tambien se ve que hay algunas intersecciones entre las subclases de STTC con las de MI, lo cual es de interes para ir identificando cuales podrian ser verdaderos positivos (i.e. que se observo un cambio en el segmento ST y que ademas tuvieron un infarto).

### Subsubclases de interes

Tambien es de interes observar si hay intersecciones entre las subsubclases que el Dr. Araiza indico, que son de STTC, y las de MI

In [260]:
sttc_interest_list = ['ISCAL', 'ISCIN', 'ISCIL', 'ISCAS', 'ISCLA']
mi_subsubclass_list = list(class_df[class_df['Superclass'] == 'MI']['Class'])
interest_list = sttc_interest_list + mi_subsubclass_list

In [261]:
interest_list

['ISCAL',
 'ISCIN',
 'ISCIL',
 'ISCAS',
 'ISCLA',
 'IMI',
 'ASMI',
 'ILMI',
 'AMI',
 'ALMI',
 'INJAS',
 'LMI',
 'INJAL',
 'IPLMI',
 'IPMI',
 'INJIN',
 'PMI',
 'INJLA',
 'INJIL']